In [1]:
# initially based on work at this link
# https://web.archive.org/web/20201031184319/https://miguelgondu.github.io/python/ai/video%20games/2018/09/04/a-tutorial-on-sc2reader-events-and-units.html 

import pandas as pd
import sc2reader
from sc2reader.scripts import sc2json
import json
import datetime
from sc2reader.engine.plugins import SelectionTracker, APMTracker
from sc2reader.factories.plugins.replay import toJSON

sc2reader.engine.register_plugin(SelectionTracker())
sc2reader.engine.register_plugin(APMTracker())

In [2]:
# Load one of professional replays
replay = sc2reader.load_replay(
    '../data/replays/SpawningTool/Pro/page1/2021-09-05-zserral-vs-pshowtime.SC2Replay',
    load_level=4)

In [10]:
events_list = replay.events

In [11]:
# How many events are there?
len(events_list)

151759

In [14]:
# extract the name and frame of each event


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_str_prefix',
 'frame',
 'load_context',
 'name',
 'pid',
 'second',
 'sid',
 'type',
 'uid']

In [39]:
def dict_replace_value(d):
    x = {}
    for k, v in d.items():
        if isinstance(v, dict):
            v = dict_replace_value(v)
        elif isinstance(v, list):
            v = list_replace_value(v)
        elif 'sc2reader.utils.DepotFile' in str(type(v)):
            v = None
        elif isinstance(v, bytes):
            v = None
        elif 'Attribute' in str(type(v)):
            v = None
        x[k] = v
    return x


def list_replace_value(l):
    x = []
    for e in l:
        if isinstance(e, list):
            e = list_replace_value(e)
        elif isinstance(e, dict):
            e = dict_replace_value(e)
        elif 'sc2reader.utils.DepotFile' in str(type(e)):
            e = None
        elif isinstance(e, bytes):
            e = None
        elif 'Attribute' in str(type(e)):
            e = None
        x.append(e)
    return x

a = dict_replace_value(replay.raw_data)

In [40]:
with open('replay_json.json', 'w', encoding='utf-8') as file:
    json.dump(a, file, indent=4, separators=(',', ': '))

TypeError: Object of type ChatEvent is not JSON serializable

In [3]:
# Create a list of event_names for each event in replay.events
event_names = set([event.name for event in replay.events])
events_of_type = {name: [] for name in event_names}
for event in replay.events:
    events_of_type[event.name].append(event)

print(events_of_type.keys())

dict_keys(['ControlGroupEvent', 'PlayerSetupEvent', 'GetControlGroupEvent', 'BasicCommandEvent', 'UpdateTargetPointCommandEvent', 'UnitDoneEvent', 'ProgressEvent', 'TargetPointCommandEvent', 'PlayerLeaveEvent', 'UpdateTargetUnitCommandEvent', 'UserOptionsEvent', 'UnitPositionsEvent', 'ChatEvent', 'AddToControlGroupEvent', 'PlayerStatsEvent', 'CameraEvent', 'UpgradeCompleteEvent', 'UnitTypeChangeEvent', 'SelectionEvent', 'UnitDiedEvent', 'UnitOwnerChangeEvent', 'UnitBornEvent', 'UnitInitEvent', 'SetControlGroupEvent', 'TargetUnitCommandEvent'])


## Locating Relevant Data
We are interested in the events that *should* be instigated by hotkey action. If a hotkey is available, and it is reasonable, assume that a hotkey was used. Eg. New players may use mouse clicks to start a building, but most long time players regardless of skill level would use a hotkey. It should feed into our data about what sequences of actions are required.

Relevant events would possibly be:
#### DataCommandEvent(CommandEvent): > USE
Should be tracked as this would relate to a hotkey:

    Extends :class:`CommandEvent`

    DataCommandEvent are recorded when ever a player issues a command that has no target. Commands
    like Burrow, SeigeMode, Train XYZ, and Stop fall under this category.

    Note that like all CommandEvents, the event will be recorded regardless
    of whether or not the command was successful.

#### 'SetControlGroupEvent' > USE
Appears to be potentially related to hotkey action:

    Extends :class:`ControlGroupEvent`

    This event does a straight forward replace of the current control group contents
    with the player's current selection. This event doesn't have masks set.

#### 'BasicCommandEvent' > USE
Appears to be additional command event:

    Extends :class:`CommandEvent`

    This event is recorded for events that have no extra information recorded.

    Note that like all CommandEvents, the event will be recorded regardless
    of whether or not the command was successful.

#### 'GetControlGroupEvent' > Use
Is activate through control group hotkey

From the docs:

    Extends :class:`ControlGroupEvent`

    This event replaces the current selection with the contents of the control group.
    The mask data is used to limit that selection to units that are currently selectable.
    You might have 1 medivac and 8 marines on the control group but if the 8 marines are
    inside the medivac they cannot be part of your selection.

#### 'UpdateTargetUnitCommandEvent' > IGNORE
Appears that Update events are automatic events when something in the game state changes and not when the player interacts.

From the docs:

    Extends :class:`TargetUnitCommandEvent`

    This event is generated when a TargetUnitCommandEvent is updated, likely due to
    changing the target unit. It is unclear if this needs to be a separate event
    from TargetUnitCommandEvent, but for flexibility, it will be treated
    differently.

    One example of this event occurring is casting inject on a hatchery while
    holding shift, and then shift clicking on a second hatchery.
     
#### 'UpdateTargetPointCommandEvent' > IGNORE
Again appears to relate to events that are generated as a secondary result of user interaction, and not relate to direct actions.

From the docs:

    Extends :class: 'TargetPointCommandEvent'

    This event is generated when the user changes the point of a unit. Appears to happen
    when a unit is moving and it is given a new command. It's possible there are other
    instances of this occurring.


#### 'SelectionEvent' > USE
May be useful. Might be hard to differentiate between hotkey and mouse interactions. Include for now

    Selection events are generated when ever the active selection of the
    player is updated. Unlike other game events, these events can also be
    generated by non-player actions like unit deaths or transformations.

    Starting in Starcraft 2.0.0, selection events targeting control group
    buffers are also generated when control group selections are modified
    by non-player actions. When a player action updates a control group
    a :class:`ControlGroupEvent` is generated. 

#### 'CameraEvent' > USE
> Hotkey input could be inferred by finding repeated jumps to the exact same hotspot and assuming these could only be reproduced so accurately by hotkey.

Docs directly state that it cannot determine if this was from hokey. Probably best to be ignored and see if this interaction is available elsewhere:

    Camera events are generated when ever the player camera moves, zooms, or rotates.
    It does not matter why the camera changed, this event simply records the current
    state of the camera after changing.

#### 'AddToControlGroupEvent' > USE
This would have to be a hotkey event

    Extends :class:`ControlGroupEvent`

    This event adds the current selection to the control group.

#### 'TargetUnitCommandEvent' > USE
Most of these would be hotkey events. Might be possible to determine from the type if they are or are not

    Extends :class:`CommandEvent`

    This event is recorded when ever a player issues a command that targets a unit.
    The location of the target unit at the time of the command is also recorded. Commands like
    Chronoboost, Transfuse, and Snipe fall under this category.

    Note that like all CommandEvents, the event will be recorded regardless
    of whether or not the command was successful.

#### 'UnitInitEvent' > USE
Not part of game.py. Records the initial state of units at the beginning of the game.

    The counter part to :class:`UnitDoneEvent`, generated by the game engine when a unit is
    initiated. This applies only to units which are started in game before they are finished.
    Primary examples being buildings and warp-in units.

#### 'UnitTypeChangeEvent' > USE
Might be duplicated elsewhere, but seems to be useful because most transformations would be instigated by hotkey.

    Generated when the unit's type changes. This generally tracks upgrades to buildings (Hatch,
    Lair, Hive) and mode switches (Sieging Tanks, Phasing prisms, Burrowing roaches). There may
    be some other situations where a unit transformation is a type change and not a new unit.

#### 'TargetPointCommandEvent' > USE
These abilities would almost certainly be called by hotkey

    Extends :class:`CommandEvent`

    This event is recorded when ever a player issues a command that targets a location
    and NOT a unit. Commands like Psistorm, Attack Move, Fungal Growth, and EMP fall
    under this category.

    Note that like all CommandEvents, the event will be recorded regardless
    of whether or not the command was successful.

#### 'UpgradeCompleteEvent' > USE for now
> Attempt to back-calculate start time by using build time. Handle Protoss by tracking if ChronoBoost ability was used on the building?

    Generated when a player completes an upgrade.

Reading the code __[here](https://github.com/ggtracker/sc2reader/blob/upstream/sc2reader/engine/plugins/apm.py)__ defines the following three events as affecting APM. APM measures the rate of human interaction with the game, and so it would be fair to assume that all necessary player interaction would be captured in these three:

1. handleControlGroupEvent
2. handleSelectionEvent
3. handleCommandEvent

Extract all of these events and print to see what information is contained.

#### Compare to APM
As there is an existing plugin for it, use the calculated APM to compare with the number of actions that this program has identified. May be useful (at least during construction) to inform whether all user actions are being captured.

In [4]:
interesting_events_list = ['DataCommandEvent', 'SetControlGroupEvent', 
        'BasicCommandEvent', 'SelectionEvent', 'AddToControlGroupEvent', 'TargetUnitCommandEvent', 'UnitTypeChangeEvent', 'TargetPointCommandEvent', 'GetControlGroupEvent', 'UnitInitEvent',
        'UnitTypeChangeEvent', 'CameraEvent', 'UpgradeCompleteEvent']

Things that seem to be missing from this list:

- Starting a unit construction - *Found in **UnitInitEvent** including buildings, maybe upgrades too*
- Starting a building construction - *Same as unit construction*
- Starting an upgrade - *Might have to use ending time and then calculate backwards to start time based on build speed, could be hard for Protoss with Chrono which modifies build speed

In [5]:
# build a list of attributes for each event
# can be used as a directory for navigating replay.events
event_attribute_dict = {}
for list_i in interesting_events_list:
    event_attribute_dict[list_i] = []
    try:
        event_i = events_of_type[list_i][0] # only use first value for now
    except:
        continue
    # NOTE - Use this loop later to loop through each event if necessary?
    # Might not be necessary if each even is identical with different values
    # for i, event_i in enumerate(events):
    
    # dir(event_i) returns a list containing attribute names
    event_dir = dir(event_i)

    # loop through elements remove any __???__ type strings to get
    # meaningful names likely to lead to value
    # use event_dir.name as key
    for attr_name in event_dir:
        
        # create dict key for attr_name linked to a blank dictionary to be
        # filled in after upcoming loop
        # event_attribute_dict[attr_name] = {}
        if (attr_name[0] != '_') & (attr_name != 'name'):
            event_attribute_dict[list_i].append(attr_name)

# print a list of all event keys to .json
# print(event_attribute_dict)
with open('../info/event_attribute_dict.json', 'w', encoding="utf-8") as file:
    json.dump(event_attribute_dict, file, indent=4, separators=(',', ': '))



In [6]:
event_attribute_dict['DataCommandEvent']

[]

In [7]:
# extract this information from event of type datacommandevent as a test
for att in event_attribute_dict['DataCommandEvent']:
    val = getattr(events_of_type['DataCommandEvent'][0], att)
    print(att, val)


---

#### Converting MS File Time to Human Readable Datetime

We need to convert MS File Time to something more useful
Conversion will be based on method found in __[this link](https://gist.github.com/Mostafa-Hamdy-Elgiar/9714475f1b3bc224ea063af81566d873)__

In [15]:
d = datetime.datetime(
    year=2021,
    month=9,
    day=5
)
print(int(d.timestamp()))

1630814400


In [17]:
filetime = 132750822404142771

In [18]:
EPOCH_AS_FILETIME = 116444736000000000  # January 1, 1970 as MS file time
HUNDREDS_OF_NANOSECONDS = 10000000

In [20]:
datetime.datetime.utcfromtimestamp((filetime - EPOCH_AS_FILETIME) / HUNDREDS_OF_NANOSECONDS)

datetime.datetime(2021, 9, 2, 18, 50, 40, 414277)